# Azure Machine Learning - Automated (Auto) ML

## Use case
Wyznaczanie ceny ubezpieczenia osoby dla której posiadamy następujące dane:
- region zamieszkania - *region*
- płeć - *sex*
- wiek - *age*
- BMI (body mass index) - *bmi*
- osoba paląca/nie paląca - *smoker*
- liczba dzieci - *children*
Do tego celu należy stworzyć model regresji w oparciu o cechy wymienione powyżej oraz ceny ubezpieczeń.

## Stworzenie zasobu w Azure
W celu rozpoczęnia pracy z AutoML należy stworzyć zasób za pośrednictwem strony [portal.azure.com](https://portal.azure.com).
1. Otwórz stronie [portal.azure.com](https://portal.azure.com)
1. Wybierz opcji ***Create resource***
1. Wyszukaj ***Machine learning***
1. Utwórz zasób

## Stworzenie Workspace'a
Kolejnym krokiem jest stworzenia Workspace'a, czyli przestrzeni, w której będziemy przeprowadzali nasze eksperymenty.
1. Udaj się na stronę [ml.azure.com](https://ml.azure.com/)
2. Wybierz opcję ***Create new workspace***


## Dodanie Dataset'u
Przed przystąpieniem do pracy nad modelem musimy na początku wgrać dane do naszego Workspace'a.
W tym celu należy przejść do zakładki Dataset, która jest widoczna na pasku nawigacyjnym po lewej stronie.
1. Wybierz ***Create dataset > From local files***
1. Nadaj nazwę dla zbioru danych i wybierz typ Tabular - aby wgrać plik CSV
1. Naciśnij ***Browse*** i znajdź plik z danymi na dysku
1. Następnie dostosuj ustawienia związane z formatem pliku
1. Na końcu zatwierdź

## Praca nad Notebookiem 

### Inicjalizacja Workspace'a
Pierwszy etap pracy z notebookiem AML to inicjalizacja przestrzeni w której będziemy pracować, w tym celu będziemy musieli się zalogować do konta Azure. Po przez Workspace mamy dostęp między innymi do zapisanych zbiorów danych bądź też wytrenowanych wcześniej modeli.

In [1]:
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.21.0


### Wczytanie danych
Na początku musimy wczytać zbiór danych, który został uprzednio dodany do workspace'u.

In [3]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

# Get Workspace defined in by default config.json file
ws = Workspace.from_config()

In [5]:
dataset = Dataset.get_by_name(ws, name='insurance')
full_df = dataset.to_pandas_dataframe()
full_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [6]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
age         1338 non-null int64
sex         1338 non-null object
bmi         1338 non-null float64
children    1338 non-null int64
smoker      1338 non-null object
region      1338 non-null object
charges     1338 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


### Przygotowanie zbioru danych
Kolejnym krokiem będzie przygotowanie zbioru danych tak aby lepiej nadawał się on do wytrenowania modelu regresji. W tym celu dokonano nastepujących modyfikacji:
- zmiana formatu danych dla kolumny *smoker* 
    - przed: string 'yes'/'no'
    - po: int 1/0
    - nowa nazwa kolumny: *smoker_num*
- zmiana formatu danych dla kolumny *sex* 
    - przed: string 'female'/'male'
    - po: int 1/2
    - nowa nazwa kolumny: *sex_num*
- zmiana formatu danych dla kolumny *region* 
    - przed: string - 4 różne wartości
    - po: int z przedzialu 1-4
    - nowa nazwa kolumny: *region_num*

In [7]:
full_df['smoker_num'] = full_df['smoker'].apply(lambda x: 1 if x == 'yes' else 0)
full_df['sex_num'] = full_df['sex'].apply(lambda x: 1 if x == 'male' else 2)
region_mapping = {region: idx + 1 for idx, region in enumerate(full_df['region'].unique())}
full_df['region_num'] = full_df['region'].apply(lambda x: region_mapping[x])
num_features = [name for name, type in full_df.dtypes.iteritems() if type == 'int64' or type == 'float64']

cleared_dataset = Dataset.Tabular.register_pandas_dataframe(full_df[num_features],  (ws.get_default_datastore(), 'insurance/'), 'insurance_cleaned')
full_df.info()
print(f'\n\nNumerical features: {num_features}\nwill be used for regression.')

Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to insurance//15cf5ba5-458e-4f2c-9d97-d83d245238f7/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
age           1338 non-null int64
sex           1338 non-null object
bmi           1338 non-null float64
children      1338 non-null int64
smoker        1338 non-null object
region        1338 non-null object
charges       1338 non-null float64
smoker_num    1338 non-null int64
sex_num       1338 non-null int64
region_num    1338 non-null int64
dtypes: float64(2), int64(5), object(3)
memory usage: 104.7+ KB


Numerical features: ['age', 'bmi', 'children', 'charges', 'smoker_num', 'sex_num', 'region_num']
will be used for regression.


### Zapisanie nowego zbioru danych
Tak przygotowany, nowy zbiór danych został zapisany do Workspace'a tak aby mógł posłużyć do kolejnych eksperymentów.
Kolumny w nowym zbiorze danych:
- age
- bmi
- children
- charges
- smoker_num
- sex_num
- region_num

### Statysyki opisowe nowego zbioru danych
Przedstawienie statystyk nowe opisowych nowego zbioru danych

In [8]:
cleaned_dataset = Dataset.get_by_name(ws, name='insurance_cleaned')
cleaned_df = cleaned_dataset.to_pandas_dataframe()
cleaned_df.describe()

,age,bmi,children,charges,smoker_num,sex_num,region_num
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265,0.204783,1.494768,2.484305
std,14.049960,6.098187,1.205493,12110.011237,0.403694,0.500160,1.104885
min,18.000000,15.960000,0.000000,1121.873900,0.000000,1.000000,1.000000
25%,27.000000,26.296250,0.000000,4740.287150,0.000000,1.000000,2.000000
50%,39.000000,30.400000,1.000000,9382.033000,0.000000,1.000000,2.000000
75%,51.000000,34.693750,2.000000,16639.912515,0.000000,2.000000,3.000000
max,64.000000,53.130000,5.000000,63770.428010,1.000000,2.000000,4.000000


### Podział zbioru
Wydzielenie ze zbioru zawierającego wszystkie rekordy 2 podzbiory:
- treningowy - służący do nauczenia modelu
- testowy - służący do sprawdzenia skuteczności działania nauczonego modelu

In [9]:
train_dataset, test_dataset = cleaned_dataset.random_split(0.7, seed=1)

# Use Pandas DF only to check the data
train_dataset_df = train_dataset.to_pandas_dataframe()
test_dataset_df = test_dataset.to_pandas_dataframe()

print(f"train_dataset_df rows: {len(train_dataset_df.index)}")
print(f"test_dataset_df rows: {len(test_dataset_df.index)}")

train_dataset_df rows: 965
test_dataset_df rows: 373


In [10]:
train_dataset_df.head()

,age,bmi,children,charges,smoker_num,sex_num,region_num
0,19,27.900,0,16884.92400,1,2,1
1,28,33.000,3,4449.46200,0,1,2
2,33,22.705,0,21984.47061,0,1,3
3,31,25.740,0,3756.62160,0,2,2
4,37,27.740,3,7281.50560,0,2,3


### Wczytanie jednostki obliczeniowej
Przed przystąpieniem do trenowania modelu należy na początku wczytać zasób obliczeniowy, na którym uruchomiony zostanie proces uczenia.

In [11]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ComputeTarget.list(ws)

[AmlCompute(workspace=Workspace.create(name='insurance', subscription_id='23996c96-391e-45b6-a56c-4d58cc49b763', resource_group='lab-automl'), name=salarySF, id=/subscriptions/23996c96-391e-45b6-a56c-4d58cc49b763/resourceGroups/lab-automl/providers/Microsoft.MachineLearningServices/workspaces/insurance/computes/salarySF, type=AmlCompute, provisioning_state=Failed, location=northeurope, tags=None),
 {
   "id": "/subscriptions/23996c96-391e-45b6-a56c-4d58cc49b763/resourceGroups/lab-automl/providers/Microsoft.MachineLearningServices/workspaces/insurance/computes/Compute-StandardDS2v2",
   "name": "Compute-StandardDS2v2",
   "location": "northeurope",
   "tags": null,
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "northeurope",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "STANDARD_DS2_V2",
       "applications": [
         {
           "di

In [12]:
# Define remote compute target to use
# Further docs on Remote Compute Target: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-remote

# Choose a name for your cluster.
amlcompute_cluster_name = "Compute-StandardDS2v2"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'ComputeInstance':
     found = True
     print('Found existing training cluster.')
     # Get existing cluster
     # Method 1:
     aml_remote_compute = cts[amlcompute_cluster_name]
     # Method 2:
     # aml_remote_compute = ComputeTarget(ws, amlcompute_cluster_name)
    
if not found:
     print('Creating a new training cluster...')
     provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D13_V2", # for GPU, use "STANDARD_NC12"
                                                                 #vm_priority = 'lowpriority', # optional
                                                                 max_nodes = 20)
     # Create the cluster.
     aml_remote_compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
aml_remote_compute.wait_for_completion(show_output = True)

Found existing training cluster.
Checking cluster status...

Running


In [13]:
from azureml.train import automl

# List of possible primary metrics is here:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric
    
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('regression')

['normalized_root_mean_squared_error',
 'r2_score',
 'spearman_correlation',
 'normalized_mean_absolute_error']

### Konfiguracja procesu uczenia
Kolejnym krokiem jest skonfigurowanie procesu uczenia. W tym miejsu ustawiamy między innymi:
- typ procesu uczenia: regresja/klasyfikacja
- metrykę, która ma podlegać minimalizacji
- maksymalny czas trwania procesu uczenia
- wcześniej wczytany lub przygotowany zbiór treningowy
- nazwę kolumny, którą wyznaczać ma nasz model regresji
- ilość walidacji podzbiorów wykorzystanych w walidacji krzyżowej

In [24]:
import logging
import os
# You can provide additional settings as a **kwargs parameter for the AutoMLConfig object
# automl_settings = {
#     "whitelist_models": 'XGBoostClassifier'
# }

from azureml.train.automl import AutoMLConfig

project_folder = './automl'
os.makedirs(project_folder, exist_ok=True)

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='regression',
                             primary_metric='r2_score',
                             experiment_timeout_minutes=30,                            
                             training_data=train_dataset,
                             label_column_name="charges",
                             n_cross_validations=5,
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log='automated_ml_errors.log',
                             verbosity=logging.INFO,
                             path=project_folder
                             # **automl_settings
                             )

# PaperCut?: Why is drop_column_names only supported by Time Series Forecast? - If used for classification, you get:
# drop_column_names= ['EmployeeCount','EmployeeNumber','Over18','StandardHours'], # Clean up dataset by dropping not needed columns
# WARNING - Received unrecognized parameter: drop_column_names ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours']
# In documentation it doesn't state that it is only supported for Forecast...:
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py

# Explanation of Settings: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings

# AutoMLConfig info on: 
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig

### Uruchomienie uczenia
Następnie należy uruchomić wybieranie modelu zgodnie z wcześniej ustawioną konfiguracją.

In [25]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "classif-automl-remote-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)

print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

classif-automl-remote-01-26-2021-18
Running on remote.
No run_configuration provided, running on Compute-StandardDS2v2 with default configuration
Running on remote compute: Compute-StandardDS2v2
Parent Run ID: AutoML_59ba7ab4-e9ec-4d94-960e-d59b2ca57c03

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:

In [26]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Czas wykonywania eksperymetu/wybieranie najlepszego modelu

In [27]:
import time
import datetime as dt

run_details = run.get_details()

# Like: 2020-01-12T23:11:56.292703Z
end_time_utc_str = run_details['endTimeUtc'].split(".")[0]
start_time_utc_str = run_details['startTimeUtc'].split(".")[0]
timestamp_end = time.mktime(datetime.strptime(end_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())
timestamp_start = time.mktime(datetime.strptime(start_time_utc_str, "%Y-%m-%dT%H:%M:%S").timetuple())

parent_run_time = timestamp_end - timestamp_start
print('Run Timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (parent_run_time))

Run Timing: --- 2192.0 seconds needed for running the whole Remote AutoML Experiment ---


In [28]:
pip show azureml-sdk

Name: azureml-sdk
Version: 1.21.0
Summary: Microsoft Azure Machine Learning Python SDK
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages
Requires: azureml-train, azureml-core, azureml-pipeline, azureml-train-automl-client, azureml-dataset-runtime
Required-by: 
Note: you may need to restart the kernel to use updated packages.


### Pobranie najlepszego modelu
Z wcześniej przeprowadzonego eksperymentu pobieramy teraz model, który uzyskał najlepsze wyniki.

In [29]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

ModuleNotFoundError: No module named 'azureml.automl.runtime._ml_engine.featurizer_suggestion'

### Przygotowanie zbioru testowego
Aby sprawdzić skuteczność działania naszego modelu musimy na początku wyodrębnić z niego kolumnę ***charges***, tak aby nie mogła posłużyć on jako jedna ze zmiennych dla naszego modelu.

In [ ]:
import pandas as pd

#Remove Label/y column
if 'charges' in test_dataset_df.columns:
    y_test_df = test_dataset_df.pop('charges')

x_test_df = test_dataset_df

### Wyznaczenie ceny ubezpieczenia dla rekordów w zbiorze testowym


In [ ]:
# Try the best model
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])

In [ ]:
y_predictions.shape

### Metryki dla zbioru testowego
Metryki przedstawiające skuteczność działania nauczonego modelu regresji służącego do wyznaczania ceny ubezpieczenia

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

print(f'R^2 (coefficient of determination) regression score:  {r2_score(y_test_df, y_predictions)}')
print(f'Mean squared error regression loss:  {mean_squared_error(y_test_df, y_predictions)}')
print(f'Mean absolute percentage error regression loss:  {mean_absolute_percentage_error(y_test_df, y_predictions)}')
